In [ ]:
# Variatin of CNN Extractor
# Given temporal matrix of w(window size) long, global kernel sweeps the entire matrix while local kernel only passes the last wl time steps of the matrix
# This experimenet was conducted based on the assumption that global trends span across longer time than local features which might be obsolete if they are from far past

In [18]:
import os
import tensorly as tl
import torch
import torch.nn.functional as F
from torch import nn
import numpy as np
import statistics
import sys
import torch.optim

In [19]:
sys.path.append("home/seyunkim/Documents/time_series_data/PEMS-SF")

In [20]:
# data preprocessing 1 batch : (963,144)
temp1 = []
temp2 = []
temp3 = []

data1 = []
data2 = []
data3 = []
start = 0
indx = 0
with open("/home/seyunkim/Documents/time_series_data/PEMS-SF/PEMS_train") as f:
    for i, line in enumerate(f):
        if i==1:
            line = f.readline()
            line = line[1:len(line)-1]
            indx = line.find(';',indx,-1)
            temp1 = line[start:indx]
            data1 = np.asarray(temp1.split(' '))
            start = indx+1
            temp1 = []
            while line.find(';',start,-1)!=-1:
                indx = line.find(';',start,-1)
                temp1 = line[start:indx]
                tempnp = np.asarray(temp1.split(' '))
                data1 = np.vstack((tempnp, data1))
                temp1=[]
                start=indx+1
            temp1 = line[start:-1]
            data1 = np.vstack((data1,np.asarray(temp1.split(' '))))
        if i==2:
            start = 0
            indx = 0
            line = f.readline()
            line = line[1:len(line)-1]
            indx = line.find(';',indx,-1)
            temp2 = line[start:indx]
            data2 = np.asarray(temp2.split(' '))
            start = indx+1
            temp2 = []
            while line.find(';',start,-1)!=-1:
                indx = line.find(';',start,-1)
                temp2 = line[start:indx]
                tempnp = np.asarray(temp2.split(' '))
                data2 = np.vstack((tempnp, data2))
                temp2=[]
                start=indx+1
            temp2 = line[start:-1]
            data2 = np.vstack((data2,np.asarray(temp2.split(' '))))
        if i==3:
            start = 0
            indx = 0
            line = f.readline()
            line = line[1:len(line)-1]
            indx = line.find(';',indx,-1)
            temp3 = line[start:indx]
            data3 = np.asarray(temp3.split(' '))
            start = indx+1
            temp3 = []
            while line.find(';',start,-1)!=-1:
                indx = line.find(';',start,-1)
                temp3 = line[start:indx]
                tempnp = np.asarray(temp3.split(' '))
                data3 = np.vstack((tempnp, data3))
                temp3=[]
                start=indx+1
            temp3 = line[start:-1]
            data3 = np.vstack((data3,np.asarray(temp3.split(' '))))


In [15]:
data2.shape

(963, 144)

In [21]:
train_tensor=torch.FloatTensor(np.ndarray.astype(data1,'float32').reshape(1,1,963,144))
val_tensor=torch.FloatTensor(np.ndarray.astype(data2,'float32').reshape(1,1,963,144))
test_tensor=torch.FloatTensor(np.ndarray.astype(data3,'float32').reshape(1,1,963,144))

In [2]:
# class Lambda(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.func = func
    
#     def forward(self, x):
#         return self.func(x)

In [3]:
# tl.set_backend('numpy')
# tensor_data = torch.FloatTensor(np.random.rand(1,1,30,500))
# split = tensor_data.shape[3]//4

In [4]:
# train_tensor = tensor_data[:,:,:,0:split*2]
# test_tensor = tensor_data[:,:,:,split*2:split*3]
# val_tensor = tensor_data[:,:,:,split*3:tensor_data.shape[3]]

In [96]:
class CNN_Extractor(nn.Module):
    def __init__(self, w, lw, R, gKernel, lKernel):
        super().__init__()
        #self.local_kernel = local_kernel
        #self.global_kernel =  global_kernel
        self.w = w
        self.R = R
        self.gKernel = gKernel
        self.lKernel = lKernel
        self.conv2d_local = nn.Conv2d(1,1,(self.R,self.lKernel)) # 30x1
        self.conv2d_global = nn.Conv2d(1,1,(self.R,self.gKernel)) # 30x5
        self.fc = nn.Linear(in_features=w-gKernel+lw-lKernel+2, out_features=self.R) 
            # Change this number of out_features to forecast longer time
            
    def forward(self, x):
        _local = self.conv2d_local(x[:,:,:,w-lw-1:-1])
        _global = self.conv2d_global(x)
        x = torch.cat((_local,_global),3)
        x = self.fc(x)
        x = torch.tanh(x)
        return x

In [102]:
# Hyperparameters
learning_rate = 1e-3
train_len = train_tensor.shape[3]
val_len = val_tensor.shape[3]
# test_len = test_tensor.shape[3]
epochs = 100
w = 100 # window size for global kernel
gK = 50 # i
lw = 50
lK = 5
# Create Model
cnn_extractor = CNN_Extractor(w,lw, 963,gK,lK)
loss_fnc = nn.MSELoss()
optimizer = torch.optim.SGD(cnn_extractor.parameters(), lr=learning_rate, momentum=0.9)


In [105]:
# Train and evaluation
for epoch in range(epochs):
    losses = torch.FloatTensor([0])
    for i in range(train_len-w):
        train_input = train_tensor[:,:,:,i:i+w] #input Rx1
        train_target = torch.transpose(train_tensor[:,:,:,i+w:i+w+1],2,3)
        
        cnn_extractor.train()
        y_pred = cnn_extractor(train_input)
        loss = loss_fnc(train_target, y_pred)
        losses = torch.hstack((losses, loss))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if epoch%2==0:
        cnn_extractor.eval()
        val_losses = torch.FloatTensor([0])
        for i in range(val_len-w):
#             cnn_extractor.eval()
            val_input = val_tensor[:,:,:,i:i+w]
            val_target = torch.transpose(val_tensor[:,:,:,i+w:i+w+1],2,3)
            y_val = cnn_extractor(val_input)
            val_loss = loss_fnc(val_target, y_val)
            val_losses = torch.dstack((val_losses, val_loss))
        val_avg = torch.mean(val_losses)
        train_avg = torch.mean(losses)
        print(f'Epoch: {epoch}, Loss: {train_avg}\t\tEpoch: {epoch}, Val-loss: {val_avg}')



Epoch: 0, Loss: 0.008826905861496925		Epoch: 0, Val-loss: 0.007321014069020748
Epoch: 2, Loss: 0.008783906698226929		Epoch: 2, Val-loss: 0.007298081647604704
Epoch: 4, Loss: 0.008753555826842785		Epoch: 4, Val-loss: 0.007275958079844713
Epoch: 6, Loss: 0.00872502289712429		Epoch: 6, Val-loss: 0.0072539858520030975
Epoch: 8, Loss: 0.008697469718754292		Epoch: 8, Val-loss: 0.007232161238789558
Epoch: 10, Loss: 0.008670530281960964		Epoch: 10, Val-loss: 0.0072104488499462605
Epoch: 12, Loss: 0.008644013665616512		Epoch: 12, Val-loss: 0.0071888272650539875
Epoch: 14, Loss: 0.008617807179689407		Epoch: 14, Val-loss: 0.0071672843769192696
Epoch: 16, Loss: 0.008591845631599426		Epoch: 16, Val-loss: 0.007145813666284084
Epoch: 18, Loss: 0.008566087111830711		Epoch: 18, Val-loss: 0.007124410942196846
Epoch: 20, Loss: 0.00854050274938345		Epoch: 20, Val-loss: 0.007103077135980129
Epoch: 22, Loss: 0.008515072986483574		Epoch: 22, Val-loss: 0.0070818099193274975
Epoch: 24, Loss: 0.0084897866472601

In [54]:
def test_evaluate(data, w, comment):
    test_losses = torch.tensor([0])
    for i in range(data.shape[3]-w):
        test_data = data[:,:,:,i:i+w]
        test_target = torch.transpose(data[:,:,:,i+w:i+w+1],2,3)
        prediction = cnn_extractor(test_data)

        test_loss = loss_fnc(test_target, prediction)
        test_losses = torch.dstack((test_loss, test_losses))
    
    test_avg = torch.mean(test_losses)
    print(f'Comment: {comment}\nTest Loss : {test_avg}')

In [55]:
test_evaluate(test_tensor, w, "w=100, w = 100, gK = 50, lw = 50,lK = 5")

Comment: w=100, w = 100, gK = 50, lw = 50,lK = 5
Test Loss : 0.007040831726044416


In [ ]:
test_evaluate(test_tensor, w, "w=100, gk=50")

In [26]:
test_evaluate(test_tensor, w, "w=100")

Comment: w=100
Test Loss : 0.005983165465295315


In [23]:
test_evaluate(test_tensor, w, "real data")

Comment: real data
Test Loss : 0.00761840958148241


In [9]:
test_evaluate(test_tensor, "fixed eval")

Comment: fixed eval
Test Loss : 0.08689478039741516


In [10]:
test_evaluate(test_tensor, "dawon")

Comment: dawon
Test Loss : 0.08689478039741516


In [11]:
test_evaluate(test_tensor, "Global+local")

Comment: Global+local
Test Loss : 0.08689478039741516


In [12]:
test_evaluate(test_tensor, "Larger dataset")

Comment: Larger dataset
Test Loss : 0.08689478039741516


In [13]:
test_evaluate(test_tensor, "Without ReLu")

Comment: Without ReLu
Test Loss : 0.08689478039741516


In [14]:
# # Evaluate with test data
# test_losses = torch.tensor([0])
# for i in range(test_len-5):
#     test_data = test_tensor[:,:,:,i:i+5]
#     test_target = torch.transpose(test_tensor[:,:,:,i+5:i+6],2,3)
#     prediction = cnn_extractor(test_data)
    
#     test_loss = loss_fnc(test_target, prediction)
#     test_losses = torch.dstack((test_loss, test_losses))

# test_avg = torch.mean(test_losses)
# print(f'Test Loss : {test_avg}')
    